In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as soup_
import re
import pandas as pd
from tabulate import tabulate
import os
import time
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.support.ui import Select
import time
import random 
import string

In [2]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

c:\users\sneha rawat\appdata\local\programs\python\python38\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
df_mmv_means_raw = pd.read_csv('ref_means_mmvy_local_0225.csv')
df_mmv_means_raw = df_mmv_means_raw.loc[1451:]
df_mmv_means_raw.head()

,make,model,variant,variant_id,fuel_type,state_id,Kms,year,mean_mmvy
1451,DATSUN,REDI GO,T (O),3202,Petrol,1,40000,2016,150456.20590
1452,HONDA,ACCORD,2.4 VTI L MT,2196,Petrol,1,130000,2007,96261.54412
1453,HONDA,AMAZE,1.2 EMT I VTEC,406,Petrol,1,40000,2016,306605.32650
1454,HYUNDAI,CRETA,1.6 SX (O) CRDI,4532,Diesel,1,52000,2016,860804.34480
1455,HYUNDAI,ELITE I20,MAGNA 1.4 CRDI,471,Diesel,1,78000,2014,369029.86210


In [4]:
def state_name_conv(state_id):
    if state_id == 1:
        return 'New Delhi'
    elif state_id == 10:
        return 'Ahmedabad'
    elif state_id == 14:
        return 'Mumbai'
    elif state_id == 18:
        return 'Hyderabad'
    elif state_id == 19:
        return 'Bangalore'
    elif state_id == 20:
        return 'Chennai'

In [5]:
def Kms_conv(Kms):
    if Kms <=5000:
        return '5,000'
    elif Kms <= 10000:
        return '10,000'
    elif Kms <= 15000:
        return '15,000'   
    elif Kms <= 20000:
        return '20,000'
    elif Kms <= 25000:
        return '25,000'
    elif Kms <= 30000:
        return '30,000'
    elif Kms <= 35000:
        return '35,000'
    elif Kms <= 40000:
        return '40,000'
    elif Kms <= 50000:
        return '50,000'
    elif Kms <= 60000:
        return '60,000'
    elif Kms <= 70000:
        return '70,000'
    elif Kms <= 80000:
        return '80,000'
    elif Kms <= 90000:
        return '90,000'
    elif Kms <= 100000:
        return '1,00,000'
    elif Kms <= 110000:
        return '1,10,000'
    elif Kms <= 120000:
        return '1,20,000'
    else:
        return '1,20,000'

In [6]:
df_mmv_means = df_mmv_means_raw.copy()
df_mmv_means['state_name'] = df_mmv_means.state_id.apply(state_name_conv)
df_mmv_means['Kilometers_driven'] = df_mmv_means.Kms.apply(Kms_conv)

df = df_mmv_means.drop(columns= ['state_id','fuel_type'])

df = df[['make','model','year','variant','state_name','Kilometers_driven']]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27234 entries, 1451 to 28684
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   make               27234 non-null  object
 1   model              27234 non-null  object
 2   year               27234 non-null  int64 
 3   variant            27234 non-null  object
 4   state_name         27234 non-null  object
 5   Kilometers_driven  27234 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.2+ MB


In [8]:
scrape_list = []

In [9]:
def scrape(yr, make, ml, vt, odo, cy):
    try:
        PROXY = "##$%@!"
        binary = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe')
        chrome_options = webdriver.FirefoxOptions()
        chrome_options.add_argument('--proxy-server=%s' % PROXY)
        driver = webdriver.Firefox(executable_path = 'geckodriver', firefox_binary = binary, options = chrome_options)
        driver.get("https://www.google.com")
        url = "https://www.cardekho.com/sell-used-car"
        driver.implicitly_wait(10)
        driver.get(url)
        time.sleep(3)
        driver.find_element_by_xpath('//*[@id="mmvData"]').click()
        years = driver.find_elements_by_tag_name('li')
        for year in years:
            if year.text == yr: 
                year.click()
                break
        brands = driver.find_elements_by_tag_name("li")
        for brand in brands:
            if fuzz.WRatio(make.lower(), brand.text.lower()) >= 95:
                brand.click()
                break
        driver.find_elements_by_class_name('sellvariant marginTopZero')
        models =  driver.find_elements_by_tag_name("li")
        for model in models:
            if fuzz.WRatio(ml.lower(), model.text.lower()) >= 95:
                model.click()
                break
        driver.find_elements_by_class_name('sellvariant')
        variants =  driver.find_elements_by_tag_name("li")
        for variant in variants:
            if fuzz.WRatio(vt.lower(), variant.text.lower()) >= 95:
                variant.click()
                break
        driver.find_elements_by_class_name('sellKilometers')        
        kms =  driver.find_elements_by_tag_name("li")
        for km in kms:
            if km.text == odo:
                km.click()
                break
        owners =  driver.find_elements_by_tag_name("li")
        for owner in owners:
            if owner.text == 'First':
                owner.click()
                break
        driver.find_elements_by_tag_name("sellAllCity clearfix")
        cities = driver.find_elements_by_tag_name("span")
        for city in cities:
            if city.text == str(cy):
                city.click()
                break     
        min_val = driver.find_element_by_xpath('.//span[@id="recPrizeSldr-min-val"]')
        max_val = driver.find_element_by_xpath('.//span[@id="recPrizeSldr-max-val"]')   
        min_val = min_val.text
        max_val = max_val.text
        driver.quit()
        return min_val, max_val
        
    except:
        driver.quit()

In [ ]:
for index, item_ in df.iterrows():
    try:    
        min_val, max_val = scrape(str(item_[2]), str(item_[0]), str(item_[1]),  str(item_[3]),  str(item_[5]), str(item_[4]))     

        scrape_dict = {'make':item_[0],
                       'model':item_[1],
                       'year':item_[2],
                       'trim':item_[3],
                       'state':item_[4],
                       'odo':item_[5],
                       'Min': min_val,
                       'Max': max_val
                      }   
    except:
        scrape_dict = {'make':item_[0],
                       'model':item_[1],
                       'year':item_[2],
                       'trim':item_[3],
                       'state':item_[4],
                       'odo':item_[5],
                       'Min':'-',
                       'Max':'-'
                      }
        
    print(scrape_dict)
    scrape_list.append(scrape_dict)
    time.sleep(2)

{'make': 'DATSUN', 'model': 'REDI GO', 'year': 2016, 'trim': 'T (O)', 'state': 'New Delhi', 'odo': '40,000', 'Min': '-', 'Max': '-'}
{'make': 'HONDA', 'model': 'ACCORD', 'year': 2007, 'trim': '2.4 VTI L MT', 'state': 'New Delhi', 'odo': '1,20,000', 'Min': '-', 'Max': '-'}
{'make': 'HONDA', 'model': 'AMAZE', 'year': 2016, 'trim': '1.2 EMT I VTEC', 'state': 'New Delhi', 'odo': '40,000', 'Min': '-', 'Max': '-'}
{'make': 'HYUNDAI', 'model': 'CRETA', 'year': 2016, 'trim': '1.6 SX (O) CRDI', 'state': 'New Delhi', 'odo': '60,000', 'Min': '-', 'Max': '-'}
{'make': 'HYUNDAI', 'model': 'ELITE I20', 'year': 2014, 'trim': 'MAGNA 1.4 CRDI', 'state': 'New Delhi', 'odo': '80,000', 'Min': 'Rs.3,80,800', 'Max': 'Rs.4,08,000'}
{'make': 'HYUNDAI', 'model': 'ELITE I20', 'year': 2014, 'trim': 'SPORTZ 1.4', 'state': 'New Delhi', 'odo': '80,000', 'Min': 'Rs.4,03,200', 'Max': 'Rs.4,32,000'}
{'make': 'HYUNDAI', 'model': 'EON', 'year': 2011, 'trim': 'ERA', 'state': 'New Delhi', 'odo': '90,000', 'Min': '-', 'Max

{'make': 'MARUTI', 'model': 'WAGON R', 'year': 2007, 'trim': 'VXI BS III', 'state': 'Mumbai', 'odo': '1,20,000', 'Min': '-', 'Max': '-'}
{'make': 'MARUTI', 'model': 'ALTO', 'year': 2012, 'trim': 'LXI', 'state': 'Mumbai', 'odo': '80,000', 'Min': '-', 'Max': '-'}
{'make': 'FORD', 'model': 'FIGO', 'year': 2012, 'trim': '1.4 EXI DURATORQ ', 'state': 'Chennai', 'odo': '1,10,000', 'Min': '-', 'Max': '-'}
{'make': 'HYUNDAI', 'model': 'I10', 'year': 2008, 'trim': 'ERA', 'state': 'Chennai', 'odo': '1,20,000', 'Min': '-', 'Max': '-'}
{'make': 'MARUTI', 'model': 'ALTO 800', 'year': 2013, 'trim': 'LXI', 'state': 'Chennai', 'odo': '70,000', 'Min': '-', 'Max': '-'}
{'make': 'HONDA', 'model': 'CITY ZX', 'year': 2007, 'trim': 'GXI', 'state': 'Bangalore', 'odo': '1,20,000', 'Min': '-', 'Max': '-'}
{'make': 'HYUNDAI', 'model': 'I10', 'year': 2013, 'trim': 'MAGNA 1.2 KAPPA2', 'state': 'Bangalore', 'odo': '70,000', 'Min': 'Rs.3,02,400', 'Max': 'Rs.3,24,000'}
{'make': 'HYUNDAI', 'model': 'I10', 'year': 201

{'make': 'TATA', 'model': 'INDICA VISTA', 'year': 2012, 'trim': 'VX QUADRAJET', 'state': 'New Delhi', 'odo': '1,10,000', 'Min': 'Rs.1,10,000', 'Max': 'Rs.1,25,000'}
{'make': 'TATA', 'model': 'TIAGO', 'year': 2017, 'trim': '\tXZA 1.2 REVOTRON', 'state': 'New Delhi', 'odo': '30,000', 'Min': 'Rs.1,63,000', 'Max': 'Rs.1,77,000'}
{'make': 'TOYOTA', 'model': 'COROLLA', 'year': 2006, 'trim': 'H5 1.8 E', 'state': 'New Delhi', 'odo': '1,20,000', 'Min': '-', 'Max': '-'}
{'make': 'TOYOTA', 'model': 'ETIOS', 'year': 2011, 'trim': 'V', 'state': 'New Delhi', 'odo': '90,000', 'Min': '-', 'Max': '-'}
{'make': 'VOLKSWAGEN', 'model': 'VENTO', 'year': 2012, 'trim': 'TRENDLINE DIESEL', 'state': 'New Delhi', 'odo': '1,10,000', 'Min': '-', 'Max': '-'}
{'make': 'TATA', 'model': 'MANZA', 'year': 2010, 'trim': 'AURA SAFIRE BS IV', 'state': 'New Delhi', 'odo': '1,00,000', 'Min': '-', 'Max': '-'}
{'make': 'MARUTI', 'model': 'SX4', 'year': 2011, 'trim': 'VXI', 'state': 'New Delhi', 'odo': '90,000', 'Min': 'Rs.1,4

In [ ]:
pd.DataFrame(scrape_list).to_csv('cardekhoscrape.csv')

In [ ]:
scrape_list